This is a second dataset I would like to use with the ucs dataset.

In [1]:
import pandas as pd
df_debris = pd.read_csv('../data/original/satcat.csv')

# Rename every column now so we don't have to deal with it later.
debris_mapping = {
    'OBJECT_NAME': 'object_name',
    'OBJECT_ID': 'object_id',          # International Designator (e.g. 1998-067A)
    'NORAD_CAT_ID': 'norad_id',        # The Key!
    'OBJECT_TYPE': 'object_type',      # PAY, R/B, DEB
    'OPS_STATUS_CODE': 'ops_status',   # +, -, P, B, S, X, D, ?
    'OWNER': 'owner',
    'LAUNCH_DATE': 'launch_date',
    'LAUNCH_SITE': 'launch_site',
    'DECAY_DATE': 'decay_date',
    'PERIOD': 'period_minutes',
    'INCLINATION': 'inclination_degrees',
    'APOGEE': 'apogee_km',
    'PERIGEE': 'perigee_km',
    'RCS': 'rcs',                      # Radar Cross Section (Size)
    'DATA_STATUS_CODE': 'data_status', # NEA, EAR, etc.
    'ORBIT_CENTER': 'orbit_center',    # EA (Earth), EL (Elliptical), etc.
    'ORBIT_TYPE': 'orbit_type'         # IMP (Impact), ORB (Orbit), etc.
}

df_debris.rename(columns=debris_mapping, inplace=True)

current_junk = df_debris[df_debris['decay_date'].isnull()].copy()

print(f"Total objects currently in orbit: {len(current_junk)}")
print("Breakdown by Type:")
print(current_junk['object_type'].value_counts())

Total objects currently in orbit: 32687
Breakdown by Type:
object_type
PAY    17562
DEB    12672
R/B     2401
UNK       52
Name: count, dtype: int64


In [2]:
current_junk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32687 entries, 4 to 67263
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   object_name          32687 non-null  object 
 1   object_id            32687 non-null  object 
 2   norad_id             32687 non-null  int64  
 3   object_type          32687 non-null  object 
 4   ops_status           16016 non-null  object 
 5   owner                32687 non-null  object 
 6   launch_date          32687 non-null  object 
 7   launch_site          32687 non-null  object 
 8   decay_date           0 non-null      object 
 9   period_minutes       32072 non-null  float64
 10  inclination_degrees  32072 non-null  float64
 11  apogee_km            32072 non-null  float64
 12  perigee_km           32072 non-null  float64
 13  rcs                  14799 non-null  float64
 14  data_status          940 non-null    object 
 15  orbit_center         32687 non-null  obje